In [7]:
from tensorboard.backend.event_processing.event_file_loader import EventFileLoader
from tensorboard.util import tensor_util
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import plotly.graph_objects as go
def get_file_list(log_folder, prefix):
    # Define the path to the log folder
    files = os.listdir(log_folder)
    list_logs = []
    for filename in files:
        if filename.startswith(prefix):
            list_logs.append(filename)
    return list_logs

In [20]:
ppo_log_folder = "../log/evaluation"
ppo_adapt_log_folder = "../log/adapt_evaluation"

ppo_logs = get_file_list(ppo_log_folder, "ppo_")
ppo_adapt_logs = get_file_list(ppo_adapt_log_folder, "ppo_")

In [21]:
output_df = pd.DataFrame(columns=['step', 'algo', 'ticker', 'eval_reward'])

# Load the log folder using EventFileLoader
for log in ppo_adapt_logs:
    col_name = log.split("_")[:2]
    algo = "_".join(col_name)
    ticker = log.split("_")[2]
    # join string in a list with seperator _

    logdir = f"{ppo_adapt_log_folder}/{log}"
    # print(logdir)
    logdir = f"{logdir}/{os.listdir(logdir)[0]}"
    loader = EventFileLoader(logdir)

    # Extract scalar data for a specific tag
    tag = 'metric/eval_reward'
    scalars = []
    for event in loader.Load():
        for v in event.summary.value:
            if v.tag == tag:
                scalars.append(
                    {
                        'step': event.step, 
                        'eval_reward': tensor_util.make_ndarray(v.tensor).item(),
                        'algo': algo,
                        'ticker': ticker
                    }
                )
    output_df = output_df.append(scalars, ignore_index=True)

for log in ppo_logs:
    col_name = log.split("_")[:2]
    algo = col_name[0]
    ticker = col_name[1]
    # join string in a list with seperator _

    logdir = f"{ppo_log_folder}/{log}"
    logdir = f"{logdir}/{os.listdir(logdir)[0]}"
    loader = EventFileLoader(logdir)

    # Extract scalar data for a specific tag
    tag = 'metric/eval_reward'
    scalars = []
    for event in loader.Load():
        for v in event.summary.value:
            if v.tag == tag:
                scalars.append(
                    {
                        'step': event.step, 
                        'eval_reward': tensor_util.make_ndarray(v.tensor).item(),
                        'algo': algo,
                        'ticker': ticker
                    }
                )
    output_df = output_df.append(scalars, ignore_index=True)

In [22]:
group_df = output_df.groupby(['step', 'algo']).aggregate({'eval_reward': ['mean', 'std']}).unstack()
group_df.columns = group_df.columns.droplevel(0)
group_df = group_df.swaplevel(1, 0, axis=1)

In [23]:
group_df

algo,ppo,ppo_adapt,ppo,ppo_adapt
,mean,mean,std,std
step,,,,
0,-3.268186,-4.636866,12.559125,29.237377
1,-2.717558,5.426318,14.153357,30.062379
2,-3.119352,6.080032,14.860891,31.161762
3,-3.297253,5.119932,14.502327,31.902287
4,-3.252488,5.065502,13.537841,32.251543
5,-2.239325,6.374222,15.219113,31.820510


In [29]:
import plotly.graph_objects as go
fig = go.Figure()
fig.update_layout(width=1000, height=500)
fig.update_layout(title=f"Mean evaluation reward over every stock in VNALL", title_x=0.5)
fig.update_xaxes(title_text="Number of learning steps before evaluation")
fig.update_yaxes(title_text="Mean reward")
fig.add_trace(go.Scatter(x=group_df['ppo'].index*1000, y=group_df['ppo']['mean'],
                    mode='lines+markers',
                    name='PPO (random weight)'))
fig.add_trace(go.Scatter(x=group_df['ppo_adapt'].index*1000, y=group_df['ppo_adapt']['mean'],
                    mode='lines+markers',
                    name='PPO-adapt'))
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

In [10]:
fig = go.Figure()
fig.update_layout(width=1000, height=500)
fig.update_layout(title=f"Mean evaluation reward across VNALL stocks starting with 'H'", title_x=0.5)
fig.update_xaxes(title_text="Number of learning steps before evaluation")
fig.update_yaxes(title_text="Mean reward")

fig.add_trace(go.Scatter(x=group_df['ppo_noexplore'].index*1000, y=group_df['ppo_noexplore']['mean'],
                    mode='lines+markers',
                    name='PPO no exploration'))

fig.add_trace(go.Scatter(x=group_df['ppo_explore0.1'].index*1000, y=group_df['ppo_explore0.1']['mean'],
                    mode='lines+markers',
                    name='PPO explore 10% time steps'))

fig.add_trace(go.Scatter(x=group_df['ppo_explore0.2'].index*1000, y=group_df['ppo_explore0.2']['mean'],
                    mode='lines+markers',
                    name='PPO explore 20% time steps'))

fig.add_trace(go.Scatter(x=group_df['ppo_explore0.3'].index*1000, y=group_df['ppo_explore0.3']['mean'],
                    mode='lines+markers',
                    name='PPO explore 30% time steps'))
# legend position

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

In [3]:
data = pd.read_csv("../temp/maml.csv")
data['MA_Value'] = data['Value'].rolling(window=20).mean()

In [8]:
data.columns

Index(['Wall time', 'Step', 'Value', 'MA_Value'], dtype='object')

In [18]:
fig = go.Figure()
fig.update_layout(width=1000, height=500)
fig.update_layout(title=f"MAML++ Outer loss", title_x=0.5)
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Outer loss")
fig.add_trace(go.Scatter(x=data['Step'], y=data['MA_Value'],
                    mode='lines+markers',
                    line=dict(width=3, color="salmon"),
                    name='Moving Average'))
fig.add_trace(go.Scatter(x=data['Step'], y=data['Value'],
                    mode='lines',
                    line=dict(width=1, color="lightslategray"),
                    name='Outer loss'))
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()